### Dataset
The real part is unbelievably large compared to the imaginary part, this is to test if we can only use the real part.

In [1]:
import torch
import math
import random
import numpy as np

In [2]:
data = torch.load('/home/matthias/Github/Hough-Transformations/data/qspace_orig01.pt')

In [ ]:
data.shape

In [ ]:
def get_mag(point):
    #return torch.norm(point)
    return math.sqrt(point[0]**2 + point[1]**2)

# test 
print(get_mag(data[0][0,0,0]))

Experiment: Caclulate $\frac{voxel_{real}}{voxel_{magnitudes}}$ and check if it is approximately the same.

In [ ]:
voxel = data[np.random.randint(0, 500)]
voxel.shape

In [ ]:
print('min-real:', voxel[:,:,:,0].min().item())
print('max-real:', voxel[:,:,:,0].max().item())

print('min-imag:', voxel[:,:,:,1].min().item())
print('max-imag:', voxel[:,:,:,1].max().item())

In [ ]:
proportions = []
proportions_imag = []

for i in range(voxel.shape[0]):
    for j in range(voxel.shape[1]):
        for k in range(voxel.shape[2]):
            point = voxel[i,j,k]
            magnitude = math.sqrt(point[0]**2 + point[1]**2)
            proportions.append(point[0]/magnitude)
            proportions_imag.append(point[1]/magnitude)

print('min:', min(proportions).item())
print('max:', max(proportions).item(), end='\n\n')

for i in range(5):
    print(proportions[random.randint(0, len(proportions))].item())

print('')
print('ones :', proportions.count(1))
print('other:', len(proportions) - proportions.count(1), end='\n\n')

print('mean of real proportions:', (sum(proportions)/len(proportions)).item())
print('mean of imag proportions:', (sum(proportions_imag)/len(proportions_imag)).item())

In [ ]:
# now the same over the 500 voxels in the dataset
proportions = []
proportions_imag = []

# only 100
for voxel in data[:100]:
    for i in range(voxel.shape[0]):
        for j in range(voxel.shape[1]):
            for k in range(voxel.shape[2]):
                point = voxel[i,j,k]
                magnitude = math.sqrt(point[0]**2 + point[1]**2)
                proportions.append(point[0]/magnitude)
                proportions_imag.append(point[1]/magnitude)

print('min:', min(proportions).item())
print('max:', max(proportions).item(), end='\n\n')

print('ones :', proportions.count(1))
print('other:', len(proportions) - proportions.count(1), end='\n\n')

print('mean of real proportions:', (sum(proportions)/len(proportions)).item())
print('mean of imag proportions:', (sum(proportions_imag)/len(proportions_imag)).item())

In [ ]:
print('-ones:', proportions.count(-1))

In [ ]:
ones = proportions.count(1)
minus_ones = proportions.count(-1)
all = len(proportions)

In [ ]:
print('ones:', ones)
print('-ones:', minus_ones)
print('other:', all - ones - minus_ones)
print('total:', all)

In [ ]:
# find values that are not 1 and -1
not_ones = [x for x in proportions if x != 1 and x != -1]
print('not ones:', len(not_ones))

In [ ]:
# some not_ones
for i in range(5):
    print(not_ones[random.randint(0, len(not_ones))].item())

In [ ]:
# get lowest and highest values
print('min:', min(not_ones).item())
print('max:', max(not_ones).item())

# get nearest to 0
print('nearest to 0:', min(not_ones, key=lambda x: abs(x)).item())

---

In [ ]:
# iterate over all in "data" (500) but only use the center of the voxel, 3x3x3
proportions_real = []
proportions_imag = []
start = 10
end = 24
status = 0

for voxel in data:
    for i in range(start, end):
        for j in range(start, end):
            for k in range(start, end):
                point = voxel[i,j,k]
                magnitude = math.sqrt(point[0]**2 + point[1]**2)
                proportions_real.append(point[0]/magnitude)
                proportions_imag.append(point[1]/magnitude)
    # prpgress bar 0-500
    status += 1
    print('status:', status, end='\r')

print('--- results ---')
print('shape of data:', data.shape, end = '\n\n')
      
print('min-real:', min(proportions_real).item())
print('max-real:', max(proportions_real).item(), end='\n\n')
print('min-imag:', min(proportions_imag).item())
print('max-imag:', max(proportions_imag).item(), end='\n\n')

print('real values')
print('ones :', proportions_real.count(1))
print('-ones:', proportions_real.count(-1))
print('1--1 :', len(proportions_real) - proportions_real.count(1), end='\n\n')

print('mean of real proportions:', (sum(proportions_real)/len(proportions_real)).item())
print('mean of imag proportions:', (sum(proportions_imag)/len(proportions_imag)).item())
abs_proportions_real = [abs(x) for x in proportions_real]
print('mean of abs real proportions:', (sum(abs_proportions_real)/len(abs_proportions_real)).item(), end='\n\n')

# get nearest to 0
print('nearest to 0:', min(proportions_real, key=lambda x: abs(x)).item())


12-22

```
--- results ---
min-real: -1.0
max-real: 1.0

min-imag: -2.2625464685242974e-13
max-imag: 2.0267966938295807e-13

ones : 494560
-ones: 5440
other: 5440

mean of real proportions: 0.97824                       # WHAT
mean of imag proportions: -1.1808454438036754e-17
nearest to 0: 1.0
```

### Efficient Checking
For other proportions than 1 or -1.

In [4]:
# iterate over all in "data" (500) but only use the center of the voxel, 3x3x3
not_ones = []
status = 0
count = 0

for voxel in data:
    for i in range(voxel.shape[0]):
        for j in range(voxel.shape[1]):
            for k in range(voxel.shape[2]):
                point = voxel[i,j,k]
                magnitude = math.sqrt(point[0]**2 + point[1]**2)

                prop_real = abs((point[0]/magnitude).item())
                #print('prop_real:', prop_real)

                count += 1
                
                # if the proportion of real/mag is not 1 or -1, add to list
                if prop_real != 1:
                    #print(prop_real, 'at', i, j, k)
                    #print('real:', point[0].item())
                    #print('imag:', point[1].item())
                    not_ones.append(prop_real)
                else:
                    #print(prop_real) 
                    pass
                        

    # prpgress bar 0-500
    status += 1
    print('status:', status, end='\r')


print('--- results ---')
print('shape of data:', data.shape, end = '\n\n')

# results
print('not ones:', len(not_ones), 'which is ', len(not_ones)/count*100, '% of all points')
print('there are', count, 'points in total')


--- results ---
shape of data: (500,)

not ones: 60638 which is  0.33746834738570275 % of all points
there are 17968500 points in total


In [5]:
# mean of abs (not ones)
abs_not_ones = [abs(x) for x in not_ones]
try:
    print('mean of abs not ones:', (sum(abs_not_ones)/len(abs_not_ones)))
except ZeroDivisionError:
    print('There are none - or literally not enough!')

mean of abs not ones: 0.9999951871366353


```
--- results ---
shape of data: (500,)

not ones: 60638 which is  0.33746834738570275 % of all points
there are 17968500 points in total

mean of abs not ones: 0.9999951871366353
```